In [1]:
import torch
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import seaborn as sns 

In [3]:
from tailnflows.utils import load_raw_data, load_experiment_output_data


runs = load_raw_data('2025-01-27-vi-nuisance')

rows = []
for model, run_datas in runs.items():
    for rd in run_datas:
        rows.append({
            'model': model,
            'mean_ess': sum(rd['tst_ess']) / len(rd['tst_ess']),
            'psis_k': rd['tst_psis_k'][0],
            **rd
        })

df = pd.DataFrame(rows)
df.head()

,model,mean_ess,psis_k,seed,dim,nuisance_df,loss_label,target_label,tst_elbo,tst_ess,tst_psis_k,tst_cubo
0,mtaf,"tensor(0.0025, device='cuda:0')",1.852160,0,5,0.5,neg_elbo,heavy_nuisance,"[tensor(-3942.1030, device='cuda:0')]","[tensor(0.0025, device='cuda:0')]","(1.8521599265245292,)","[tensor(3.0255, device='cuda:0')]"
1,mtaf,"tensor(0.0024, device='cuda:0')",2.930167,0,10,0.5,neg_elbo,heavy_nuisance,"[tensor(-172255.0312, device='cuda:0')]","[tensor(0.0024, device='cuda:0')]","(2.9301670239521904,)","[tensor(-4.6628, device='cuda:0')]"
2,mtaf,"tensor(0.0010, device='cuda:0')",7.922609,0,50,0.5,neg_elbo,heavy_nuisance,"[tensor(-5129478., device='cuda:0')]","[tensor(0.0010, device='cuda:0')]","(7.922609494282649,)","[tensor(-276.8526, device='cuda:0')]"
3,mtaf,"tensor(0.0016, device='cuda:0')",1.580786,1,5,0.5,neg_elbo,heavy_nuisance,"[tensor(-4.8196e+11, device='cuda:0')]","[tensor(0.0016, device='cuda:0')]","(1.5807859966388116,)","[tensor(3.0368, device='cuda:0')]"
4,mtaf,"tensor(0.0014, device='cuda:0')",2.202037,1,10,0.5,neg_elbo,heavy_nuisance,"[tensor(-1.8665e+09, device='cuda:0')]","[tensor(0.0014, device='cuda:0')]","(2.2020366008465104,)","[tensor(-2.8524, device='cuda:0')]"


In [6]:
grouper = ['model', 'dim']
grouped = df.groupby(grouper)
agg = grouped[['mean_ess', 'psis_k']].agg(['mean', 'sem', 'std', 'count'])
# agg['p1'] = agg['mean'] + agg['sem']

best = agg['mean_ess'].sort_values(by='mean')
within = best.iloc[-1]['mean'] - best.iloc[-1]['std']

def highlight_by_mean(row, threshold=7):
    """
    Highlight the row if the 'Mean' value is above the given threshold.
    """
    if row['mean'] > threshold:
        # Highlight entire row in a light green
        return ['background-color: #dfffdf'] * len(row)
    else:
        # No highlight
        return ['background-color: white'] * len(row)
    
best.style.apply(
    highlight_by_mean, 
    axis=1, 
    threshold=within
)